In [1]:
import numpy as np

from strat import MACDandEMA

from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest


np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="15m",
    candles_to_dl=3000,
)

In [3]:
macd_strat = MACDandEMA(
    long_short="long",
    ema_length=np.arange(100, 301, 100),
    fast_length=np.arange(10, 21, 10),
    macd_below=np.array([0]),
    signal_smoothing=np.array([10]),
    slow_length=np.arange(20, 31, 10),
)

In [4]:
backtest_settings = BacktestSettings()

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=True,
    z_or_e_type=None,
)

dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([0]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([5]),
    sl_based_on_add_pct=np.array([0.1, 0.25]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

In [5]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=macd_strat,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 9
Total order settings to test: 16
Total combinations of settings to test: 144
Total candles: 3,000
Total candles to test: 432,000


In [9]:
backtest_results.sort_values(by=["total_pnl"], ascending=False).head(10)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,ending_eq,total_pnl
39,2,7,8.0,5,3,70.816,62.500,0.807,53.064,1708.160,708.160
38,2,6,8.0,5,3,70.816,62.500,0.807,53.064,1708.160,708.160
134,8,6,13.0,6,7,67.330,46.154,0.598,129.970,1673.303,673.303
135,8,7,13.0,6,7,67.330,46.154,0.598,129.970,1673.303,673.303
87,5,7,9.0,6,3,54.292,66.667,0.712,134.898,1542.918,542.918
86,5,6,9.0,6,3,54.292,66.667,0.712,134.898,1542.918,542.918
142,8,14,10.0,6,4,42.767,60.000,0.807,106.552,1427.666,427.666
130,8,2,15.0,6,9,42.080,40.000,0.626,153.944,1420.800,420.800
131,8,3,15.0,6,9,42.080,40.000,0.626,153.944,1420.800,420.800
14,0,14,8.0,3,5,33.791,37.500,0.689,87.443,1337.911,337.911


In [10]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=macd_strat,
    dos_index=7,
    ind_set_index=2,
    plot_results=True,
    logger_bool=True,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 0,
    risk_account_pct_size = 0.03,
    risk_reward = 5.0,
    sl_based_on_add_pct = 0.001,
    sl_based_on_lookback = 50,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.01,
    trail_sl_when_pct = 0.02,
)


In [8]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,2,5,357,1705825800000,2024-01-21 08:30:00,EntryFilled,1000.000,941.092,5741.621,58.908,...,5800.529,41692.6,NaN,0.139,5800.529,NaN,0.400,41526.9,2.7,42808.6
1,2,5,389,1705854600000,2024-01-21 16:30:00,StopLossFilled,970.028,970.028,NaN,NaN,...,NaN,NaN,41526.9,NaN,NaN,-29.972,NaN,NaN,NaN,NaN
2,2,5,736,1706166900000,2024-01-25 07:15:00,EntryFilled,970.028,930.936,1951.584,39.092,...,1990.676,39989.7,NaN,0.050,1990.676,NaN,1.300,39454.8,7.4,42939.4
3,2,5,849,1706268600000,2024-01-26 11:30:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.019,40752.5,NaN,NaN
4,2,5,850,1706269500000,2024-01-26 11:45:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.024,40933.3,NaN,NaN
5,2,5,851,1706270400000,2024-01-26 12:00:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.024,40964.2,NaN,NaN
6,2,5,852,1706271300000,2024-01-26 12:15:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.027,41066.8,NaN,NaN
7,2,5,857,1706275800000,2024-01-26 13:30:00,StopLossFilled,1021.451,1021.451,NaN,NaN,...,NaN,NaN,41066.8,NaN,NaN,51.423,NaN,NaN,NaN,NaN
8,2,5,941,1706351400000,2024-01-27 10:30:00,EntryFilled,1021.451,976.289,2939.278,45.162,...,2984.440,41705.0,NaN,0.072,2984.440,NaN,0.900,41335.6,5.1,43839.3
9,2,5,1015,1706418000000,2024-01-28 05:00:00,MovedTSL,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.017,42413.8,NaN,NaN
